#### 필요한 라이브러리 불러오기

In [ ]:
import cv2
import os
import numpy as np
import keras_ocr
import math
import pandas as pd
from scipy import ndimage
import skimage
from skimage import filters
from skimage.filters import threshold_otsu

#### 전처리 함수 코드

In [ ]:
# 이미지 원본 path 리스트, array를 저장할 위치, 저장할 array의 이름을 인자로 받음
def preprocessing(FILE_PATH,SAVE_PATH,SAVE_NAME):
    # 글자 지우는 모델 선언 
    pipeline = keras_ocr.pipeline.Pipeline()
    # 처리된 이미지들 받는 리스트 
    processed = []
    # 이미지 전처리 시작
    FILE_PATH = [i for i in FILE_PATH if i[-3:]=="tif"]
    
    for f in FILE_PATH:
        # 먼저 keras_ocr.tools.read로 이미지 내에 글자가 있는지 확인 
        file_path = '/root/hyundairb/DRT_CUT/'+ f
        img = keras_ocr.tools.read(f)
        dst_img = cv2.imread(f,0)
        prediction_groups = pipeline.recognize([img])
        # 글씨가 있으면 글씨 영역의 좌표를 얻기
        if(len(prediction_groups[0])!=0):
            boundary = []
            for i in range(len(prediction_groups[0])):
                x = []
                y = []
                for j in range(len(prediction_groups[0][i][1])):
                    x.append(prediction_groups[0][i][1][j][0])
                    y.append(prediction_groups[0][i][1][j][1])
                # 글씨 주변에 깔끔하지 못한 어두운 부분들도 함께 커버하기 위해 상하좌우로 11픽셀씩 더 커버하기 
                x_min = int(np.where(math.floor(min(x)-11) > 0, math.floor(min(x)-11), 0))
                x_max = math.ceil(max(x)+11)  
                y_min = int(np.where(math.floor(min(y)-11) > 0, math.floor(min(y)-11), 0))
                y_max = math.ceil(max(y)+11)
                info_list = [x_min, x_max, y_min, y_max]
                boundary.append(info_list)
            
            for i in range(len(boundary)):
                # 가로 왼쪽으로 2픽셀 전, 세로는 글자 영역 높이의 절반 되는 지점의 색깔로 글자가 있는 곳을 가림
                background = dst_img[boundary[i][2]+round(0.5*(y_max-y_min))][boundary[i][0]-2] # 픽셀
                patch = np.full((boundary[i][3]-boundary[i][2],boundary[i][1]-boundary[i][0]),background) # 가리기
                dst_img[boundary[i][2]:boundary[i][3], boundary[i][0]:boundary[i][1]] = patch
            
            # difference of gaussian으로 용접 부분 경계 추출하기 
            Dog = filters.difference_of_gaussians(dst_img, 12,12*16)
            # 찾은 경계를 90만큼 키워서 경계가 뚜렷해지도록 하기 
            size = 90
            minimum_trib = ndimage.minimum_filter(Dog, size)
            # otsu 이진화로 검은 바탕에 용접부분 경계를 흰색으로 표시하기 
            orig_sub_min = Dog - minimum_trib
            thresh = threshold_otsu(orig_sub_min)
            binary = orig_sub_min > thresh
            bool_to_pix = np.where(binary==True,255,0)
            bool_to_pix = np.array(bool_to_pix, dtype = "uint8")
            # opening으로 용접부분 경계 외에 흰색으로 표시되는 노이즈를 제거 
            rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(60,60))
            opening = cv2.morphologyEx(bool_to_pix, cv2.MORPH_OPEN, rect_kernel)
            
            # 일부 경계 추출이 안되는 경우가 있어 try문으로 작성 
            try:
                # 흰색으로 표시된 용접 부분 경계의 가장 윗부분과 가장 아랫부분의 y좌표 추출
                upper = np.max(np.where(opening==255)[0])+50
                under = np.min(np.where(opening==255)[0])-50
                # 원본 이미지에서 용접부분만 자르기 
                cut_img = dst_img[under:upper,:]
                # 이미지 내의 색상을 10가지 범위로 나누어서 10가지 색상으로 각각 채워 넣기 
                section = pd.cut(np.concatenate(cut_img).tolist(), 10, labels = [0,28,56,84,112,140,168,196,224,252])
                # 다시 이미지의 크기로 reshape 하기 
                processed_img = np.array(section).reshape(np.array(cut_img).shape)
                # 모델에 넣기 위해 이미지를 정규화하기 
                normalized_img = processed_img.astype('float32') / 255.
                # 모델에 넣기 위해 이미지를 (400,400) 크기로 만들기 
                resized_img = cv2.resize(normalized_img,(400,400),cv2.INTER_AREA)
                # 처리된 이미지 array를 processed 리스트에 append 하기 
                processed.append(resized_img)
                # 성공 메시지 출력
                print(f, ": success")
            # 만약 어떠한 문제로 용접부분 경계 추출이 되지 않는 경우 해당 파일명을 던져주고 실패 메시지 출력
            except StopIteration:
                print(f, ": failed")
                break
        # 이미지 상단에 아무런 글씨도 쓰여있지 않은 경우 글씨 제거 코드 제외 다른 코드는 동일함
        else:
            normal_img = cv2.imread(f, 0)
            Dog = filters.difference_of_gaussians(normal_img, 12,12*16)
            size = 90
            minimum_trib = ndimage.minimum_filter(Dog, size)
            orig_sub_min = Dog - minimum_trib
            thresh = threshold_otsu(orig_sub_min)
            binary = orig_sub_min > thresh
            bool_to_pix = np.where(binary==True,255,0)
            bool_to_pix = np.array(bool_to_pix, dtype = "uint8")
            rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(60,60))
            opening = cv2.morphologyEx(bool_to_pix, cv2.MORPH_OPEN, rect_kernel)
            
            try:
                upper = np.max(np.where(opening==255)[0])+50
                under = np.min(np.where(opening==255)[0])-50
                cut_img = normal_img[under:upper,:]
                section = pd.cut(np.concatenate(cut_img).tolist(), 10, labels = [0,28,56,84,112,140,168,196,224,252])
                processed_img = np.array(section).reshape(np.array(cut_img).shape)
                normalized_img = processed_img.astype('float32') / 255.
                resized_img = cv2.resize(normalized_img,(400,400),cv2.INTER_AREA)
                processed.append(resized_img)
                print(f, ": success")
            except StopIteration:
                print(f, ": failed")
                break
            np.savez_compressed(SAVE_PATH+'/'+SAVE_NAME, img = processed)